# Bottom-up Tokenization

## Develop an algorithm

### Import libraries

In [10]:
import json
import random
import re
from collections import Counter, defaultdict
import nltk
from nltk.tokenize import sent_tokenize, wordpunct_tokenize
nltk.download('punkt_tab')

from tokenizers import BertWordPieceTokenizer
import os, json, glob

# Set random seed for reproducibility
random.seed(42)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/davepipon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Load sample data

In [11]:
# Set the path to your dataset directory
path = os.path.expanduser("/Users/davepipon/Desktop/DS397 Data/coleridgeinitiative-show-us-the-data/train/*.json")

# Load training file names
files = glob.glob(path)

# Randomly select 2000 files
sample_files = random.sample(files, min(2000, len(files)))

# Read and store randomly sampled documents
sample_data = []
for path in sample_files:
    with open(path) as f:
        data = json.load(f)
        for entry in data:
            sample_data.append(entry.get("text", ""))

print("Number of sampled JSON files:", len(sample_files))
print("Number of texts stored:", len(sample_data))
print("Snippet of first doc:\n", sample_data[0][:300], "...")

Number of sampled JSON files: 2000
Number of texts stored: 37728
Snippet of first doc:
 alzheimer's disease and other types of dementia are the top cause for disabilities in later life and various types of experiments have been performed to understand the underlying mechanisms of the disease with the aim of coming up with potential drug targets. these experiments have been carried out  ...


### Pre-processing

#### Generate corpus

In [13]:
# Simple regex tokenizer (split on non-alphabetic chars)
tokens = []
for doc in sample_data:
    words = re.findall(r"\b\w+\b", doc.lower())  # lowercase for consistency
    tokens.extend(words)

# Remove tokens that has special characters or numbers
tokens = re.findall(r'[a-zA-Z]+', ' '.join(tokens))

# Generate unique tokens with frequency
token_freq = Counter(tokens)

# Sort tokens by frequency
corpus = sorted(token_freq.items(), key=lambda x: x[1], reverse=True)

print("Number of unique tokens:", len(token_freq))
print("Top 10 tokens by frequency:", corpus[:10])

Number of unique tokens: 115330
Top 10 tokens by frequency: [('the', 863626), ('of', 503701), ('and', 454964), ('in', 362744), ('to', 316623), ('a', 234964), ('for', 172715), ('is', 135841), ('that', 131452), ('with', 113159)]


#### Create initial vocabulary

In [14]:
# Extract unique list of letters from corpus as initial vocab
unique_letters = re.sub(r'[^a-zA-Z\s]', '', ''.join(tokens))
vocab = list(set(unique_letters))

print("Initial vocab size:", len(vocab))
print("Initial vocab:", vocab)

Initial vocab size: 26
Initial vocab: ['t', 'o', 'h', 'z', 'k', 'l', 'p', 'y', 'j', 'b', 'q', 'r', 'u', 'x', 's', 'a', 'c', 'w', 'g', 'f', 'e', 'd', 'v', 'i', 'm', 'n']


### Implement bottom-up tokenization

#### Byte-pairing encoding

In [15]:
# Define function to count frequency of adjacent symbol pairs
def get_pair_stats(corpus):
    pair_freq = defaultdict(int)
    for symbols, freq in corpus:
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i + 1])
            pair_freq[pair] += freq
    return pair_freq

# Define function to merge the most frequent pair in the corpus
def merge_pair(pair, corpus):
    a, b = pair
    new_corpus = []
    for symbols, freq in corpus:
        new_syms = []
        i = 0
        while i < len(symbols):
            if i < len(symbols)-1 and symbols[i] == a and symbols[i+1] == b:
                new_syms.append(a+b)   # merge
                i += 2
            else:
                new_syms.append(symbols[i])
                i += 1
        new_corpus.append((new_syms, freq))
    return new_corpus

# Main function to perform byte-pair encoding
def byte_pair_encoding(corpus, vocab, num_merges=100):
    """
    corpus: list of (word, freq) where word is a string.
    vocab: initial vocab.
    """
    # initialize corpus as list of (symbols, freq)
    corpus = [(list(word), freq) for word, freq in corpus]

    for _ in range(num_merges):
        pair_freq = get_pair_stats(corpus)
        if not pair_freq:
            break
        best_pair = max(pair_freq, key=pair_freq.get)   # most frequent
        vocab.append(''.join(best_pair))                # add to vocab
        corpus = merge_pair(best_pair, corpus)          # update corpus

    return vocab, corpus

In [16]:
# Implement BPE
num_merges = 40000
expanded_vocab, final_corpus = byte_pair_encoding(corpus, vocab, num_merges=num_merges)
print("BPE vocab size:", len(expanded_vocab))
print("BPE vocab snippet:", expanded_vocab[:50])
print(expanded_vocab[-50:])

BPE vocab size: 40026
BPE vocab snippet: ['t', 'o', 'h', 'z', 'k', 'l', 'p', 'y', 'j', 'b', 'q', 'r', 'u', 'x', 's', 'a', 'c', 'w', 'g', 'f', 'e', 'd', 'v', 'i', 'm', 'n', 'th', 'in', 're', 'the', 'on', 'at', 'an', 'er', 'en', 'al', 'st', 'or', 'ed', 'es', 'ion', 'of', 'and', 'ar', 'as', 'ic', 'it', 'ro', 'ing', 'is']
['fulfils', 'canned', 'holmst', 'holmstrom', 'mukhopadhaya', 'royer', 'samuelson', 'pursues', 'atraumatic', 'endeavours', 'unveiled', 'electra', 'subtitle', 'pointers', 'tragic', 'reallocate', 'straightforwardly', 'cranfield', 'cyril', 'negligibly', 'reusability', 'rbp', 'showcase', 'wordpiece', 'inflections', 'tricks', 'idiosyncras', 'idiosyncrasies', 'finetuning', 'segregating', 'annotating', 'distilling', 'sackett', 'ordinances', 'shorthand', 'derog', 'derogatory', 'learnings', 'giants', 'hepes', 'deionized', 'calmodulin', 'justin', 'casulli', 'whitecapping', 'morison', 'prestia', 'lipscomb', 'unclustered', 'garces']


# Compare with HuggingFace WordPiece

## Tokenization using WordPiece

In [17]:
# Define a generator to yield single-token sentences for WordPiece tokenizer
def corpus_iterator(corpus):
    for word, freq in corpus:
        for _ in range(freq):
            yield [word]  # single-token sentence

# Initialize tokenizer
tokenizer = BertWordPieceTokenizer(lowercase=True)
tokenizer.train_from_iterator(
    corpus_iterator(corpus),
    vocab_size=40000,  # Set desired vocab size
    min_frequency=1,  # Minimum frequency for a token to be included
)

print("WordPiece vocab size:", tokenizer.get_vocab_size())
print("WordPiece vocab snippet:", list(tokenizer.get_vocab().items())[:50])
print(list(tokenizer.get_vocab().items())[-50:])




WordPiece vocab size: 40000
WordPiece vocab snippet: [('arsen', 30748), ('stuart', 23765), ('fundamental', 5632), ('lsu', 36341), ('malta', 26586), ('compu', 2365), ('demonstrates', 6189), ('conclusions', 3657), ('structurally', 18853), ('paired', 7495), ('systolic', 9438), ('mumford', 34169), ('pressed', 33732), ('produc', 548), ('progra', 555), ('affection', 37267), ('gaining', 10239), ('detritus', 39497), ('##essional', 27467), ('save', 11113), ('waynes', 33731), ('cau', 5505), ('bailey', 17657), ('uncon', 7915), ('listing', 7232), ('kriging', 16353), ('##well', 6996), ('natic', 26838), ('##ept', 31600), ('##iology', 6378), ('##ulation', 589), ('tedesco', 29686), ('##be', 2450), ('##umines', 31520), ('variates', 35800), ('##gue', 18802), ('muff', 22212), ('cofound', 36945), ('ncl', 9785), ('carbonaceous', 39113), ('##yder', 16842), ('kat', 6740), ('norovirus', 30783), ('harvey', 14702), ('##as', 81), ('facs', 34755), ('suspension', 12481), ('seldom', 15014), ('##isen', 15935), ('

# Conclusion

When comparing the developed BPE and WordPiece tokenizers, the difference in token size mainly depends on their respective hyperparameters. Specifically, the BPE tokenizer’s vocabulary size is controlled by the number of merge operations ***k***, while the WordPiece tokenizer is directly constrained by the predefined vocabulary size. A key distinction is that BPE’s learned tokens are influenced by its initial vocabulary, whereas WordPiece explicitly introduces the prefix “##” to mark subword continuations. Importantly, sufficient training iterations are necessary to ensure that frequent English words are adequately represented in the vocabulary.